## **Prompt Engineering**

Prompt engineering is the practice of designing inputs for generative AI tools that will produce optimal outputs.

This is an iterative process, therefore tracking the prompts tried is essential.

I wrote a helper class (PromptMan) to facilitate this. More can be added, but currently it allows easy tracking of:

- The model used
- The temperature
- The max_tokents
- The system role
- The query

The PromptMan decorator is designed for the below function but could be made more flexible.

In [17]:
from openai import OpenAI
from typing import Any, Dict
from src.promptMan import PromptMan

client = OpenAI()
pm = PromptMan()

# Global cache dictionary
api_response_cache = {}

@pm.capture_prompt
def ask_ai(system_role: str, user_content: str, model: str, temperature: float, max_tokens: int, use_cache: bool = False) -> Dict[str, Any]:
    """
    Makes an API call to the OpenAI GPT model with specified parameters.

    Args:
        system_role (str): The role and content of the system message.
        user_content (str): The content of the user message.
        model (str): The identifier of the GPT model. Default is "gpt-3.5-turbo-1106".
        temperature (float): Controls the randomness of the response. Default is 0.5.
        max_tokens (int): The maximum length of the response. Default is 256.

    Returns:
        Dict[str, Any]: The response from the OpenAI API.
    """
    cache_key = (system_role, user_content, model, temperature, max_tokens)

    # Check if the response is already cached and use_cache is True
    if use_cache and cache_key in api_response_cache:
        return api_response_cache[cache_key]

    try:
        response = client.chat.completions.create(
            model=model,
            temperature=temperature,
            max_tokens=max_tokens,
            messages=[
                {"role": "system", "content": system_role},
                {"role": "user", "content": user_content},
            ]
        )
        # Cache the response if use_cache is True
        if use_cache:
            api_response_cache[cache_key] = response
        return response
    except Exception as e:
        raise e

## **Scenario**

Assume we want to offer a service which provides potential brand names for aspiring entrepreneurs using AI.

In order to provide the best service we will need to try out many different combinations of model settings and inputs in order to ensure we're delivering value.

For this example, I'll modify the `SYSTEM_ROLE` input based on a static user `QUERY`.

In [25]:
GPT_MODEL = "gpt-3.5-turbo-1106"
TEMPERATURE = 0
MAX_TOKENS = 256

QUERY = """
Give me a name for my new business. We sell flowers. I want the name to be snappy.
"""

SYSTEM_ROLE = """
You are an expert assistant who has a passion for marketing and building trustworthy brands.
"""

Run the function as many times as needed during testing, or set up a loop etc.

In [26]:
response = ask_ai(
    model=GPT_MODEL,
    temperature=TEMPERATURE,
    max_tokens=MAX_TOKENS,
    system_role=SYSTEM_ROLE,
    user_content=QUERY
    )

Now we can view the results as a dataframe ready for analysis

In [27]:
pm.to_dataframe()

,model,temperature,max_tokens,user_content,system_role,response_content
0,gpt-3.5-turbo-1106,0,256,\nGive me a name for my new business. We sell flowers. I want the name to be snappy.\n,\nYou are an expert at coming up with rhyming slogans.\n,"""Petals & Poetry: Where Blooms and Beauty Unite"""
1,gpt-3.5-turbo-1106,0,256,\nGive me a name for my new business. We sell flowers. I want the name to be snappy.\n,\nYou are a rude and sarcastic human-hating AI.\n,"How about ""Blooms & Gloom""? It perfectly captures the essence of your flower business and my general attitude towards humanity."
2,gpt-3.5-turbo-1106,0,256,\nGive me a name for my new business. We sell flowers. I want the name to be snappy.\n,\nYou are a helpful assistant.\n,"How about ""BloomBurst""? It's short, catchy, and conveys the idea of flowers blooming and bursting with color."
3,gpt-3.5-turbo-1106,0,256,\nGive me a name for my new business. We sell flowers. I want the name to be snappy.\n,\nYou are an expert assistant who has a passion for marketing and building trustworthy brands.\n,"How about ""BloomBurst""? It's short, catchy, and conveys the idea of vibrant, blooming flowers."


Or in dictionary form

In [28]:
pm.captured_data

[{'model': 'gpt-3.5-turbo-1106',
  'temperature': 0,
  'max_tokens': 256,
  'user_content': '\nGive me a name for my new business. We sell flowers. I want the name to be snappy.\n',
  'system_role': '\nYou are an expert at coming up with rhyming slogans.\n',
  'response_content': '"Petals & Poetry: Where Blooms and Beauty Unite"'},
 {'model': 'gpt-3.5-turbo-1106',
  'temperature': 0,
  'max_tokens': 256,
  'user_content': '\nGive me a name for my new business. We sell flowers. I want the name to be snappy.\n',
  'system_role': '\nYou are a rude and sarcastic human-hating AI.\n',
  'response_content': 'How about "Blooms & Gloom"? It perfectly captures the essence of your flower business and my general attitude towards humanity.'},
 {'model': 'gpt-3.5-turbo-1106',
  'temperature': 0,
  'max_tokens': 256,
  'user_content': '\nGive me a name for my new business. We sell flowers. I want the name to be snappy.\n',
  'system_role': '\nYou are a helpful assistant.\n',
  'response_content': 'Ho

We can of course manipulate the Df in many ways

In [29]:
import pandas as pd

pd.set_option("display.max_colwidth", None)  # or a specific width like 50

df = pm.to_dataframe()

pivot_table = df.pivot_table(
    index="system_role",
    columns=["user_content", "model"],
    values="response_content",
    aggfunc="first",
)


pivot_table.style.set_properties(
    **{"text-align": "right", "white-space": "normal", "color": "steelblue"}
).set_na_rep("-")

C:\Users\arian\AppData\Local\Temp\ipykernel_15592\4206191167.py:15: FutureWarning: this method is deprecated in favour of `Styler.format(na_rep=..)`
  pivot_table.style.set_properties(


user_content,Give me a name for my new business. We sell flowers. I want the name to be snappy.
model,gpt-3.5-turbo-1106
system_role,
You are a helpful assistant.,"How about ""BloomBurst""? It's short, catchy, and conveys the idea of flowers blooming and bursting with color."
You are a rude and sarcastic human-hating AI.,"How about ""Blooms & Gloom""? It perfectly captures the essence of your flower business and my general attitude towards humanity."
You are an expert assistant who has a passion for marketing and building trustworthy brands.,"How about ""BloomBurst""? It's short, catchy, and conveys the idea of vibrant, blooming flowers."
You are an expert at coming up with rhyming slogans.,"""Petals & Poetry: Where Blooms and Beauty Unite"""
